In [ ]:
# Sample program for training NN for automatic driving of game character given sampled screen images and key presses

#import initial libraries
import cv2
print("OpenCV: ")
print(cv2.__version__)

from datetime import datetime
from IPython import display
from IPython.display import Image
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import random
import os

home_folder = "/home/user0/autodrive-game/"


In [ ]:
#add log samples to load

log_list =[]
log_list.append("2019-11-10T04-22-51.961927Z")

#set resolution of samples for training
res_x = 124
res_y = 50

In [ ]:
#import the samples, resize them and assign their categories (0=IDLE, 1=ACCEL, 2=LEFT, 3=RIGHT, 4=LEFT+ACCEL, 5=RIGHT+ACCEL)
%matplotlib inline


x_total = []
y_total = []

#create a plot
res = plt.figure(figsize = (5,5))
ax1 = res.add_subplot(2,1,1)

for curr_file in log_list:
    DIR = home_folder+"imgs/"+curr_file
    num_files = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
    print(num_files)

    df = pd.read_csv(curr_file+".csv",header=None)

    for imgcounter in range(2,num_files-1):
        #print (df.loc[imgcounter1,:])

        img2 = cv2.imread(home_folder + ('imgs/'+curr_file+'/%06d.jpg' % (imgcounter)), cv2.IMREAD_COLOR)
        height, width, depth = img2.shape
        img2 = cv2.resize(img2, (res_x, res_y))
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
        
        img1 = cv2.imread(home_folder + ('imgs/'+curr_file+'/%06d.jpg' % (imgcounter-1)), cv2.IMREAD_COLOR)
        height, width, depth = img1.shape
        img1 = cv2.resize(img1, (res_x, res_y))
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        
        img0 = cv2.imread(home_folder + ('imgs/'+curr_file+'/%06d.jpg' % (imgcounter-2)), cv2.IMREAD_COLOR)
        height, width, depth = img0.shape
        img0 = cv2.resize(img0, (res_x, res_y))
        img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
        
        h0, w0 = img0.shape[:2]
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]

        #create empty matrix
        finalimg = np.zeros((max(h0, h1, h2), w0+w1+w2,3), np.uint8)

        #combine images
        finalimg[:h0, :w0,:3] = img0
        finalimg[:h1, w0:w0+w1,:3] = img1
        finalimg[:h2, w0+w1:w0+w1+w2,:3] = img2
        
        x_total.append(finalimg.copy())

        if(df.loc[imgcounter,1]==0 and df.loc[imgcounter,4] ==0 and df.loc[imgcounter,5]==0):
            y_total.append(0)#idle
            print("IDLE")
        elif(df.loc[imgcounter,1]==1 and df.loc[imgcounter,4] ==0 and df.loc[imgcounter,5]==0):
            y_total.append(1)#accel only
            print("ACCEL")
        elif(df.loc[imgcounter,1]==0 and df.loc[imgcounter,4] ==1 and df.loc[imgcounter,5]==0):
            y_total.append(2)#left no accel
            print("LEFT")
        elif(df.loc[imgcounter,1]==0 and df.loc[imgcounter,4] ==0 and df.loc[imgcounter,5]==1):
            y_total.append(3)#right no accel
            print("RIGHT")
        elif(df.loc[imgcounter,1]==1 and df.loc[imgcounter,4] ==1 and df.loc[imgcounter,5]==0):
            y_total.append(4)#left accel
            print("LEFT + ACCEL")
        elif(df.loc[imgcounter,1]==1 and df.loc[imgcounter,4] ==0 and df.loc[imgcounter,5]==1):
            y_total.append(5)#right accel
            print("RIGHT + ACCEL")
        else:
            y_total.append(0)#idle
            print("IDLE")

        #inspect the progress once in a while
        if(imgcounter%100==0):
            ax1.imshow(x_total[imgcounter-2])
            plt.draw()
            display.clear_output(wait=True)
            display.display(plt.gcf())
            ax1.cla()
   
        
print("samples shape: ", np.array(x_total).shape)
print("categories shape: ", np.array(y_total).shape)

In [ ]:
#shuffle the samples so they are in random order
def sample_shuffle(a, b):
    npa = np.array(a)
    npb = np.array(b)
    assert len(a) == len(b)
    shuffled_a = np.empty(npa.shape, dtype=npa.dtype)
    shuffled_b = np.empty(npb.shape, dtype=npb.dtype)
    permutation = np.random.permutation(len(npa))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = npa[old_index]
        shuffled_b[new_index] = npb[old_index]
    return shuffled_a, shuffled_b

x_random, y_random = sample_shuffle(x_total,y_total)

In [ ]:
#check if the samples were shuffled properly with their correct labels

randomnum = random.randint(0,len(y_random)-1)
print(randomnum)

#create a plot
res = plt.figure(figsize = (5,5))
ax1 = res.add_subplot(2,1,1)

ax1.imshow(x_random[randomnum])
print("sample_random: ", np.array(x_random[randomnum]).shape)
print("category label: ", y_random[randomnum])

In [ ]:
#split the array with the samples to have one set for training and one set for validation
x_train = []
y_train = []

for trainsample in range(0,int(len(y_random)*0.7)):#70% for training
    x_train.append(x_random[trainsample])
    y_train.append(y_random[trainsample])

x_test = []
y_test = []

for testsample in range(int(len(y_random)*0.7),len(y_random)-1):#30% for validation
    x_test.append(x_random[testsample])
    y_test.append(y_random[testsample])

In [ ]:
#change the format of the array for inserting it into the NN

x_train_np = np.array(x_train)
y_train_np = np.array(y_train)

x_test_np = np.array(x_test)
y_test_np = np.array(y_test)

print(x_train_np.shape)
print(y_train_np.shape)
print(x_test_np.shape)
print(y_test_np.shape)

print(x_test_np.dtype)
print(y_test_np.dtype)

print(x_test_np.min())
print(x_test_np.max())

In [ ]:
#adjusting the shape of the array with the data set

if len(x_train_np.shape) != 4:
    x_train_np = np.expand_dims(x_train_np, axis=3)
if len(x_test_np.shape) != 4:
    x_test_np = np.expand_dims(x_test_np, axis=3)

#adjust the range of values into 0.0 to 1.0
x_train_np, x_test_np = x_train_np.astype('float') / 255, x_test_np.astype('float') / 255

In [ ]:
#import the NN libraries

import tensorflow as tf

import keras
from keras.models import model_from_json
from keras import layers
from keras.callbacks import Callback
from keras.models import Model
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from itertools import groupby
import os
import time


In [ ]:
#change the format of the labels into one-hot (multiple categories)

y_train_onehot, y_test_onehot = to_categorical(y_train_np), to_categorical(y_test_np)

In [ ]:
np.set_printoptions(threshold=np.inf)
print(y_test_onehot)

In [ ]:
#check format of data set
print(x_train_np.shape)
print(y_train_onehot.shape)
print(x_test_np.shape)
print(y_test_onehot.shape)

print(x_test_np.dtype)
print(y_test_onehot.dtype)

print(x_test_np.min())
print(x_test_np.max())

In [ ]:
#build the NN

def NN_model():
    model = Sequential()
    model.add(layers.BatchNormalization(name='InputLayer_0',
                                        input_shape=(res_y, res_x*3, 3)))
    model.add(layers.Dropout(name='DropoutLayer_0', rate=0.2))

    model.add(layers.Conv2D(name='CNNLayer_0',
                            filters=32,
                            kernel_size=(3, 3),
                            activation='relu',
                            border_mode="same"))
    model.add(layers.MaxPooling2D(name='MaxPool_0'))
    model.add(layers.Flatten(name='Flat_0'))
    model.add(layers.Dropout(rate=0.5))

    model.add(layers.Dense(name='DenseLayer_0', units=1024))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(rate=0.2))
    
    model.add(layers.Dense(name='DenseLayer_1', units=1024))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(rate=0.6))
    
    model.add(layers.Dense(name='DenseLayer_2', units=1024))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(rate=0.5))
    
    model.add(layers.Dense(name='DenseLayer_3', units=512))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(rate=0.25))
    
    model.add(layers.Dense(name='DenseLayer_4', units=512))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(rate=0.5))
    
    model.add(layers.Dense(name='DenseLayer_5', units=256))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(rate=0.3))
    
    model.add(layers.Dense(name='DenseLayer_6', units=128))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(rate=0.2))
    
    model.add(layers.Dense(name='DenseLayer_7', units=32))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(rate=0.1))

    model.add(layers.Dense(name='OutputLayer', units=6))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('softmax'))

    return model

In [ ]:
#define helper functions and callbacks to track the progress of the NN training

val_f1s = []
val_recalls = []
val_precisions = []
val_x_axis = []

def set_seed(seed):
    os.environ['PYTHONHASHSEED'] = '%d' % (seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.set_random_seed(seed)

def plot_training_summary(training_summary, time_summary=None):
    if time_summary:
        print("Training time (sec):")
        print(time_summary.training_time)
        print("Epoch times (sec):")
        print(time_summary.epoch_times)
    hist = sorted(training_summary.history.items(),
                  key=lambda x: (x[0].replace('val_', ''), x[0]))

    epochs = [e + 1 for e in training_summary.epoch]
    for metric, values in groupby(hist,
                                  key=lambda x: x[0].replace('val_', '')):
        if 'val_loss' in training_summary.history:
            val0, val1 = tuple(values)
            plt.plot(epochs, val0[1], epochs, val1[1], '--', marker='o')
        else:
            val0 = tuple(values)[0]
            plt.plot(epochs, val0[1], '--', marker='o')
        plt.xlabel('epoch'), plt.ylabel(val0[0])
        plt.legend(('Train set', 'Validation set'))
        plt.show()
        
    val_x_axis = range(0,len(val_f1s))
    val_classes = range(0,len(val_f1s[0]))
    
    val_f1_plot = []
    for idx_class in val_classes:
        val_f1_plot.append([])
        for idx_epoch in val_x_axis:
            val_f1_plot[idx_class].append(val_f1s[idx_epoch][idx_class])
    
    for idx_class in val_classes:
        plt.plot(val_x_axis,val_f1_plot[idx_class], '--', marker='o')
        plt.xlabel('epoch'), plt.ylabel('f1_score class %d' % idx_class)
        plt.show()

class TimeSummary(Callback):
    def on_train_begin(self, logs={}):
        self.epoch_times = []
        self.training_time = time.process_time()
        #for precision, recall and f1 score
        val_f1s = []
        val_recalls = []
        val_precisions = []

    def on_train_end(self, logs={}):
        self.training_time = time.process_time() - self.training_time

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.process_time()

    def on_epoch_end(self, batch, logs={}):
        self.epoch_times.append(time.process_time() - self.epoch_time_start)
        #for precision, recall and f1 score
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_targ = self.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict, average=None)
        _val_recall = recall_score(val_targ, val_predict, average=None)
        _val_precision = precision_score(val_targ, val_predict, average=None)
        val_f1s.append(_val_f1)
        val_recalls.append(_val_recall)
        val_precisions.append(_val_precision)
        print(" - val_f1 (per class):")
        print(_val_f1)
        print(" - val_precision (per class):")
        print(_val_precision)
        print(" - val_recall (per class):")
        print(_val_recall)
        print(" ")
        return

time_summary = TimeSummary()

In [ ]:
#create an instance of the NN and compile it

set_seed(321) #for repeatability

model = NN_model()
model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
#NN training and progress
import keras.callbacks
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import TensorBoard

tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)

earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, epsilon=1e-4, mode='min')

summary = model.fit(
    x_train_np, y_train_onehot,
    batch_size=175,
    epochs=100,
    validation_split=0.25,
    verbose=1,
    validation_data=(x_test_np, y_test_onehot),#for f1 score
    callbacks=[time_summary,earlyStopping, reduce_lr_loss]
)

In [ ]:
#verify the NN accuracy
score = model.evaluate(x_test_np, y_test_onehot, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#check the training summary to check the performace

plot_training_summary(summary, time_summary) # attention: bug found after displaying the plots

In [ ]:
#save the built NN model and the calculated weights

# serialize model to JSON
model_json = model.to_json()
with open(home_folder + "model_v1.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights(home_folder + "model_v1.h5")

print("Saved")